In [1]:
import numpy as np
import pandas as pd
import mne
import sklearn
import matplotlib.pyplot as plot
import scipy.stats as stats
import openpyxl as op
import os
import gc

In [6]:
def preprocessing():
    # EEG 불러오기      
    try:
        eeg_data = mne.io.read_raw_edf(file_path, preload=True)
        eeg_data_raw = eeg_data.copy()
        eeg_data_info = eeg_data.info
        print("EEG를 불러왔습니다.")
        
        # Bandpass filters
        eeg_data.filter(l_freq=0.1, h_freq=90)
        
        # Notch filter apply
        eeg_data.notch_filter(freqs=[60])
        
        # 채널명 표준화
        eeg_data = eeg_data.drop_channels(eeg_data.ch_names[18:])
        eeg_data_ch_names = eeg_data.ch_names
        channel_mapping = {}
        for i in range(len(eeg_data_ch_names)):
            if eeg_data_ch_names[i] != ch_names_answer[i]:  # 차이가 있는 경우, 해당 인덱스의 값을 딕셔너리에 저장
                channel_mapping[eeg_data_ch_names[i]] = ch_names_answer[i]
        
        print(f"{variable_name}의 잘못된 채널명은 {channel_mapping} 입니다.")
        eeg_data.rename_channels(channel_mapping)
        print("채널명 표준화가 완료되었습니다.")
        
        # monopolar data로 변형
        monopolar_data = np.dot(transform_mat, eeg_data.get_data())  # monopolar 계산
        monopolar_info = mne.create_info(
            ch_names=monopolar_chs,
            sfreq=eeg_data.info['sfreq'],
            ch_types='eeg'
        )  # 새로운 info 생성
        eeg_data_monopolar = mne.io.RawArray(monopolar_data, monopolar_info)  # 새 monopolar 데이터 생성
        montage = mne.channels.make_standard_montage('standard_1020')
        eeg_data_monopolar.set_montage(montage)  # 몽타주 부여
        print("monopolar 데이터가 형성되었습니다.")
        
        # ICA 시행
        ica = mne.preprocessing.ICA(n_components=15, random_state=42)
        ica.fit(eeg_data_monopolar)
        muscle_idx, muscle_scores = ica.find_bads_muscle(eeg_data_monopolar)
        eeg_data_clean = eeg_data_monopolar.copy()
        ica.apply(eeg_data_clean, exclude=muscle_idx)
        print(f"{variable_name}의 ICA가 종료되었습니다.")
        
        # preprocessing info 저장
        info[variable_name] = (eeg_data_raw, eeg_data_info, eeg_data_clean)        

        # clean data 파일로 저장
        try: 
            eeg_data_clean.save(rf'E:\Mg_EEG\edf_subacute\{variable_name}_clean.fif', overwrite=False)
            print(f"{variable_name}의 clean data가 저장되었습니다.")
        except Exception as e:
            print(f"{variable_name}의 clean data 저장 중 오류가 발생했습니다: {str(e)}")
        
        return eeg_data_clean
    
    except Exception as e:
        print(f"{variable_name} 처리 중 오류가 발생했습니다: {str(e)}")
        return None

In [5]:
def TFR_analysis (eeg_data_clean):
    
    # bad channel 정보 확인    
    bad_channel_value = subacute_data.loc[subacute_data['name'] == name][day].values[0]
    if bad_channel_value == 'O':
        bad_channels_list = []
        print(f"{variable_name}의 bad channel은 없습니다.")
    elif bad_channel_value == 'X':
        raise ValueError("X 값이므로 오류 발생")
    else:
        bad_channels_list = [
            channel.strip() 
            for channel in bad_channel_value.split(',')
        ]
        print(f"{variable_name}의 bad channel은 {bad_channels_list} 입니다.")
    

    # bad channels 제거
    eeg_data_clean = eeg_data_clean.copy()
    eeg_data_clean.drop_channels(bad_channels_list)

    # 대역별 TFR 시행
    tfr_dict = {} # 대역별 tfr 결과 저장
    for band, (fmin, fmax) in freq_ranges.items():
        
        # TFR 분석
        print(f"{variable_name}의 {band}대역의 TFR 분석이 시작되었습니다.")
        try:
            tfr = eeg_data_clean.compute_tfr(method='multitaper', freqs=np.arange(fmin, fmax+1), n_jobs=-1, reject_by_annotation=False)
            tfr_dict[band] = tfr.data
            print(f"{variable_name}의 {band} 대역의 TFR  분석이 종료되었습니다.")
        except Exception as e:
            print(f"{variable_name}의 {band} 대역의 TFR  분석 중 오류가 발생하였습니다: {str(e)}")
        
        # TFR 저장
        saving_path = rf'E:\Mg_EEG\tfr_files_subacute\{variable_name}_{band}_tfr.h5'
        try:
            tfr.save(saving_path, overwrite=False)
            print(f"TFR 파일이 성공적으로 저장되었습니다.")     
        except Exception as e:
            print(f"TFR 파일 저장 중 오류가 발생하였습니다: {str(e)}")

        # Delete tfr for memory
        del tfr
        print(f"{name}의 데이터를 메모리에서 삭제하였습니다.")
        gc.collect()

    return tfr_dict

    

In [4]:
def power_analysis ():
    
    # Combine Gamma bands
    try:
        if 'Gamma1' in tfr_dict and 'Gamma2' in tfr_dict:
            tfr_dict['Gamma'] = np.concatenate([tfr_dict['Gamma1'], tfr_dict['Gamma2']], axis=1)
            del tfr_dict['Gamma1'], tfr_dict['Gamma2']
        print(f"{name}의 Gamma1과 Gamma2 데이터를 결합했습니다.")
    except Exception as e:
        print(f"Gamma 데이터 결합 과정에서 오류가 발생하였습니다: {str(e)}")

    # 밴드의 평균 파워 계산
    for band in ['Delta', 'Theta', 'Alpha', 'Beta', 'Gamma']:
        mean = tfr_dict[band].mean(axis=0) # 18개 채널에 대한 평균
        mean = mean.mean(axis=0) # 대역 내 주파수에 대한 평균
        mean = mean.mean(axis=0) # 5분의 파워 평균
        print(f"{name}_{day}_{band}의 평균값은 {mean} 입니다.")

        # 엑셀에 저장     
        try:
            ws_b = wb['subacute']
            name_row_offset = name_dict.get(name)
            day_row_offset = {
                'day1': 1,
                'day3': 2
            }
            band_row_offset = {
                'Delta': -13,
                'Theta': -12,
                'Alpha': -11,
                'Beta': -10,
                'Gamma': -9
            }
            
            row_b = name_row_offset * 10 + day_row_offset.get(day) * 5 + band_row_offset.get(band)        
        
            if row_b is None:
                print(f"Invalid band name: {band_name}")
                return   
            ws_b.cell(row=row_b, column=1).value = name 
            ws_b.cell(row=row_b, column=2).value = day
            ws_b.cell(row=row_b, column=3).value = band
            ws_b.cell(row=row_b, column=4).value = mean
            print(f"{variable_name}의 {band} 데이터가 저장되었습니다.")
        except Exception as e:
            print(f"{variable_name}의 {band} 데이터 저장 과정 중 오류가 발생하였습니다: {str(e)}")

In [10]:
# 기본 설정값들 모음

ch_names_answer = ['Fp1-F3', 'F3-C3', 'C3-P3', 'P3-O1', 
                'Fp2-F4', 'F4-C4', 'C4-P4', 'P4-O2', 
                'Fp1-F7', 'F7-T3', 'T3-T5', 'T5-O1', 
                'Fp2-F8', 'F8-T4', 'T4-T6', 'T6-O2', 
                'Fz-Cz', 'Cz-Pz']

bipolar_chs = ['Fp1-F3', 'F3-C3', 'C3-P3', 'P3-O1', 
                'Fp2-F4', 'F4-C4', 'C4-P4', 'P4-O2', 
                'Fp1-F7', 'F7-T3', 'T3-T5', 'T5-O1', 
                'Fp2-F8', 'F8-T4', 'T4-T6', 'T6-O2', 
                'Fz-Cz', 'Cz-Pz']

monopolar_chs = ['Fp1', 'F3', 'C3', 'P3', 'O1',
                 'Fp2', 'F4', 'C4', 'P4', 'O2',
                 'F7', 'T3', 'T4', 'T5', 'T6', 'F8',
                 'Fz', 'Cz', 'Pz']

n_bipolar = len(bipolar_chs)
n_monopolar = len(monopolar_chs)
transform_mat = np.zeros((n_monopolar, n_bipolar))

for i, mono_ch in enumerate(monopolar_chs):
    for j, bi_ch in enumerate(bipolar_chs):
        if mono_ch in bi_ch.split('-'):
            if mono_ch == bi_ch.split('-')[0]:
                transform_mat[i, j] = 1
            else:
                transform_mat[i, j] = -1

subacute_data = pd.read_excel(r"C:\Users\Brain_Science\Documents\GitHub\Mg_infusion_coma\subacute_data.xlsx", sheet_name='Sheet1')

# 각 주파수 대역의 정보
freq_ranges = {
    'Delta': (1, 4),
    'Theta': (4, 8),
    'Alpha': (8, 13),
    'Beta': (13, 30),
    'Gamma1': (30, 60),
    'Gamma2': (61, 90) # Gamma 대역은 해당하는 주파수가 너무 넓기에 2개로 나누어서 분석하고, 추후 결합함
}

# Initialize name dictionary
names = ['고정실', '김가람', '김득실', '김영현', '김충연', '민병춘', '박주연', '벌', 
            '안중훈', '윤병시', '이미우', '임석봉', '전창희', '정금례', '정용태', '이귀임', 
            '정복연', '김정한', '정광훈', '조진욱']
name_dict = {name: i+1 for i, name in enumerate(names)}

고정실,김가람,김득실,김영현,김충연,민병춘,박주연,벌,안중훈,윤병시,이미우,임석봉,전창희,정금례,정용태,이귀임,정복연,김정한,정광훈,조진욱

In [9]:
name_list = input("분석대상 이름을 입력하세요. (여러명일 경우 쉼표로 구분)").split(',')
day_list = ["day1", "day3"]
# 엑셀 위치
wb = op.load_workbook(r"C:\Users\Brain_Science\Documents\GitHub\Mg_infusion_coma\Mg_infusion_data.xlsx")
info = {}

for name in name_list:
    for day in day_list:
        # 분석대상 설정
        variable_name = f"{name}_{day}"
        print(f"분석대상은 {variable_name} 입니다.")       
        
        # 파일 존재 여부 확인
        file_path = rf"E:\Mg_EEG\edf_subacute\{variable_name}.EDF"
        if not os.path.exists(file_path):
            print(f"{variable_name}의 EDF 파일이 존재하지 않습니다.")
            continue # 파일 존재하지 않으면 다음으로 넘어감
        
        # preprocessing 시행
        eeg_data_clean = preprocessing()
        if eeg_data_clean is None:
            print(f"{variable_name}의 preprocessing에 실패했습니다.")
            continue
        
        # 대역별 TFR 시행
        tfr_dict = TFR_analysis(eeg_data_clean)

        # power analysis 시행
        power_analysis()

        print(f"{variable_name}의 분석이 완료되었습니다.")
    print(f"{name}의 분석이 완료되었습니다.")
# 엑셀에 저장
wb.save(r"C:\Users\Brain_Science\Documents\GitHub\Mg_infusion_coma\Mg_infusion_data.xlsx")

print(f"{name_list}의 분석결과를 엑셀에 저장하였습니다.")
print(f"{name_list}의 데이터 분석이 완료되었습니다.")

분석대상은 _day1 입니다.
_day1의 EDF 파일이 존재하지 않습니다.
분석대상은 _day3 입니다.
_day3의 EDF 파일이 존재하지 않습니다.
의 분석이 완료되었습니다.
['']의 분석결과를 엑셀에 저장하였습니다.
['']의 데이터 분석이 완료되었습니다.


In [16]:
# 재분석 돌리는 코드

name_list = input("분석대상 이름을 입력하세요. (여러명일 경우 쉼표로 구분)").split(',')
day_list = ["day1"]
# 엑셀 위치
# wb = op.load_workbook(r"C:\Users\Brain_Science\Desktop\new.xlsx")
info = {}

for name in name_list:
    for day in day_list:
        # 분석대상 설정
        variable_name = f"{name}_{day}"
        print(f"분석대상은 {variable_name} 입니다.")       
        
        # 파일 존재 여부 확인
        file_path = file_path = rf"E:\Mg_EEG\edf_subacute\{variable_name}.EDF"
        if not os.path.exists(file_path):
            print(f"{variable_name}의 EDF 파일이 존재하지 않습니다.")
            continue # 파일 존재하지 않으면 다음으로 넘어감
        
        # preprocessing 시행
        eeg_data_clean = preprocessing()
        if eeg_data_clean is None:
            print(f"{variable_name}의 preprocessing에 실패했습니다.")
            continue
        
        # 대역별 TFR 시행
        tfr_dict = TFR_analysis(eeg_data_clean)

        # power analysis 시행
        power_analysis()

        print(f"{variable_name}의 분석이 완료되었습니다.")
    print(f"{name}의 분석이 완료되었습니다.")
# 엑셀에 저장
# wb.save(r"C:\Users\Brain_Science\Desktop\new.xlsx")

print(f"{name_list}의 분석결과를 엑셀에 저장하였습니다.")
print(f"{name_list}의 데이터 분석이 완료되었습니다.")

분석대상은 윤병시_day1 입니다.
Extracting EDF parameters from E:\Mg_EEG\edf_subacute\윤병시_day1.EDF...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 60199  =      0.000 ...   300.995 secs...
EEG를 불러왔습니다.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 90 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 90.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 95.00 Hz)
- Filter length: 6601 samples (33.005 s)

Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 59 - 61 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
C:\Users\Brain_Science\AppData\Local\Temp\ipykernel_26012\522455965.py:41: RuntimeWarning: The data has not been high-pass filtered. For good ICA performance, it should be high-pass filtered (e.g., with a 1.0 Hz lower bound) before fitting ICA.
  ica.fit(eeg_data_monopolar)


Fitting ICA took 0.7s.
Effective window size : 10.240 (s)
Applying ICA to Raw instance
    Transforming to ICA space (15 components)
    Zeroing out 0 ICA components
    Projecting back using 19 PCA components
윤병시_day1의 ICA가 종료되었습니다.
Writing E:\Mg_EEG\edf_subacute\윤병시_day1_clean.fif
Closing E:\Mg_EEG\edf_subacute\윤병시_day1_clean.fif
[done]
윤병시_day1의 clean data가 저장되었습니다.
윤병시_day1의 bad channel은 없습니다.
윤병시_day1의 Delta대역의 TFR 분석이 시작되었습니다.


C:\Users\Brain_Science\AppData\Local\Temp\ipykernel_26012\522455965.py:52: RuntimeWarning: This filename (E:\Mg_EEG\edf_subacute\윤병시_day1_clean.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  eeg_data_clean.save(rf'E:\Mg_EEG\edf_subacute\{variable_name}_clean.fif', overwrite=False)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of  19 | elapsed:    0.1s remaining:    0.7s
[Parallel(n_jobs=-1)]: Done   8 out of  19 | elapsed:    0.2s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done  12 out of  19 | elapsed:    0.2s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done  16 out of  19 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  19 out of  19 | elapsed:    0.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 24 co

윤병시_day1의 Delta 대역의 TFR  분석이 종료되었습니다.
TFR 파일 저장 중 오류가 발생하였습니다: Destination file exists. Please use option "overwrite=True" to force overwriting.
윤병시의 데이터를 메모리에서 삭제하였습니다.
윤병시_day1의 Theta대역의 TFR 분석이 시작되었습니다.


...

윤병시_day1의 Beta 대역의 TFR  분석이 종료되었습니다.
TFR 파일 저장 중 오류가 발생하였습니다: Destination file exists. Please use option "overwrite=True" to force overwriting.
윤병시의 데이터를 메모리에서 삭제하였습니다.
윤병시_day1의 Gamma1대역의 TFR 분석이 시작되었습니다.


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of  19 | elapsed:    1.8s remaining:    7.2s
[Parallel(n_jobs=-1)]: Done   8 out of  19 | elapsed:    2.0s remaining:    2.7s
[Parallel(n_jobs=-1)]: Done  12 out of  19 | elapsed:    2.0s remaining:    1.1s
[Parallel(n_jobs=-1)]: Done  16 out of  19 | elapsed:    2.0s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done  19 out of  19 | elapsed:    2.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 24 concurrent workers.


윤병시_day1의 Gamma1 대역의 TFR  분석이 종료되었습니다.
TFR 파일 저장 중 오류가 발생하였습니다: Destination file exists. Please use option "overwrite=True" to force overwriting.
윤병시의 데이터를 메모리에서 삭제하였습니다.
윤병시_day1의 Gamma2대역의 TFR 분석이 시작되었습니다.


[Parallel(n_jobs=-1)]: Done   4 out of  19 | elapsed:    1.8s remaining:    6.9s
[Parallel(n_jobs=-1)]: Done   8 out of  19 | elapsed:    1.8s remaining:    2.6s
[Parallel(n_jobs=-1)]: Done  12 out of  19 | elapsed:    1.9s remaining:    1.1s
[Parallel(n_jobs=-1)]: Done  16 out of  19 | elapsed:    1.9s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done  19 out of  19 | elapsed:    2.0s finished


윤병시_day1의 Gamma2 대역의 TFR  분석이 종료되었습니다.
TFR 파일 저장 중 오류가 발생하였습니다: Destination file exists. Please use option "overwrite=True" to force overwriting.
윤병시의 데이터를 메모리에서 삭제하였습니다.
윤병시의 Gamma1과 Gamma2 데이터를 결합했습니다.
윤병시_day1_Delta의 평균값은 1.3368147272665006e-09 입니다.
윤병시_day1의 Delta 데이터가 저장되었습니다.
윤병시_day1_Theta의 평균값은 2.9265551831156167e-10 입니다.
윤병시_day1의 Theta 데이터가 저장되었습니다.
윤병시_day1_Alpha의 평균값은 7.435714482094691e-11 입니다.
윤병시_day1의 Alpha 데이터가 저장되었습니다.
윤병시_day1_Beta의 평균값은 2.359658619416742e-11 입니다.
윤병시_day1의 Beta 데이터가 저장되었습니다.
윤병시_day1_Gamma의 평균값은 3.6161816209363967e-12 입니다.
윤병시_day1의 Gamma 데이터가 저장되었습니다.
윤병시_day1의 분석이 완료되었습니다.
윤병시의 분석이 완료되었습니다.
['윤병시']의 분석결과를 엑셀에 저장하였습니다.
['윤병시']의 데이터 분석이 완료되었습니다.
